Big Data - TP 4 Parte 2. Sofia Ellenberg, Sophie Schulzen, Vicente Zervino 

Construccion de funciones


In [29]:
#Ejercicio 1 : 

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score


#Escribimos la funcion evalua_metodo con un modelo como argumento y los datos de entrenamiento y testeo. Luego, ajustamos el modelo con los datos.
#Por ultimo, calculamos la matriz de confusion, las curvas ROC, los valores de AUC y accuracy score. 


def evalua_metodo(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    MAT_CONF = confusion_matrix(y_test, y_pred) #Matriz de confusion
    ACCURACY = accuracy_score(y_test, y_pred) # Accuracy score
    fpr, tpr, _ = roc_curve(y_test, y_prob)  # Curva roc
    AUC_ROC = auc(fpr, tpr) # AUC

#Matriz de confgusion 
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')
    plt.title('Matriz de Confusión')
    plt.xlabel('Predicción')
    plt.ylabel('Real')
    
#Curva ROC
    plt.subplot(1, 2, 2)
    plt.plot(fpr, tpr, lw=2, color= 'blue', label=f'Curva ROC (AUC = {AUC_ROC:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Falsos Positivos')
    plt.ylabel('Verdaderos Positivos')
    plt.title('Curva ROC')
    plt.legend(loc="lower right")
    plt.show()

#Creamos un dccionario que devuelva las metricas.
    METRICAS = {
        'confusion_matrix': MAT_CONF,
        'accuracy': ACCURACY,
        'roc_auc': AUC_ROC,
        'roc_curve': (fpr, tpr)
    }
    
    return metrics


In [30]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [31]:
 #Ejercicio 2: 

#Ahora escribimos la funcion cross_validation que haga cross validation con k iteraciones. En cad auna llamara a la funcion del inciso anterior pero para las k partiicones. 
#Como argumento recibira, el modelo, el valor de k y x e y como dataset. #Hacemos las particiones usando la funcion Kfold.

def cross_validation(model, k, X, y):
    KFOLD = KFold(n_splits=k, shuffle=True, random_state=42)
    ACCURACY_SCORES = []
    AUC_SCORES = []

    for train_index, test_index in KFOLD.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        METRICAS = evalua_metodo(model, X_train, y_train, X_test, y_test)
        ACCURACY_SCORES.append(metrics['accuracy'])
        AUC_SCORES.append(metrics['roc_auc'])
    
    accuracy_promedio = np.mean(ACCURACY_SCORES)
    auc_promedio = np.mean(AUC_SCORES)
    
    print(f'Accuracy Promedio: {accuracy_promedio:.4f}')
    print(f'AUC-ROC Promedio: {auc_promedio:.4f}')
    
    return accuracy_promedio, auc_promedio

Ejercicio 3
Escribimos la funcion evalua_config que reciba una lista de configuraciones de hiperparámetros, donde cada configuración será un diccionario de Python.


In [33]:
#Ejercicio 3 
def evalua_config(model_class, hyperparameter_configs, k, X, y):
    mejor_configuracion = None #Tenemos que inicializar la variable para luego poder almacenar la mejor configuracion y el menor error.
    error_minimo = float('inf') 

#Como dice la consgna iteramos hiperparámetros
    for configuracion in hyperparameter_configs:
        print(f"Evaluar la configuracion: {configuracion}")
        
        modelo = model_class(**configuracion)
#Obtenemos el error promedio con cross validation usamdo el ECM
        errores = []
        for train_index, test_index in KFOLD.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            error = mean_squared_error(y_test, y_pred)
            errores.append(error)
        
        error_promedio = np.mean(errores)


        if error_promedio < error_minimo: #Por si encontramos un error menor 
            error_minimo = error_promedio
            mejor_configuracion = configuracion

#Printiamos la mejor configuración que hayamos encontrado y el menor error
    print("La mejor configuración es:", mejor_configuracion)
    print("Error mínimo:", error_minimo)
    
#La funcion nos devuelve la configuracion que genere menor error.
    return mejor_configuracion

Ejercicio 4:
Escribimos una funcion llamada evalua_multiples_metodos. Usamos la funcion evalua_config para optimizar el λ de la regularizacion.

Obtenemos una tabla donde las columas son las metricas (de la fc evalua_metodo) y las filas son los modelos (con su configuracion de hiperparametros asociada) que corrimos antes. 


In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import KFold
from statistics import mean
import pandas as pd

In [40]:
#Ejercicio 4

#Usamos las funciones de los ejercicios anteriores. 
# Función para evaluar múltiples métodos. 

def evalua_multiples_metodos(X_train, y_train, X_test, y_test):
    results = []
    
# Regresión Logística
    logreg_hyperpar = [
        {'penalty': 'l1', 'C': 0.1},
        {'penalty': 'l2', 'C': 0.01},
        {'penalty': 'none'}
    ]
    mejor_configuracion_logreg = evalua_config(LogisticRegression, logreg_hyperpar, 5, X_train, y_train)
    logreg_model = LogisticRegression(**mejor_configuracion_logreg)
    logreg_metricas = evalua_metodo(logreg_model, X_train, y_train, X_test, y_test)
    results.append({'Modelo': 'Reg Logística', 'Configuración': mejor_configuracion_logreg, **logreg_metricas})
    
    # Análisis de Discriminante Lineal
    ADL_model = LinearDiscriminantAnalysis()
    ADL_metricas = evalua_metodo(ADL_model, X_train, y_train, X_test, y_test)
    results.append({'Modelo': 'Análisis Discriminante Lineal', 'Configuración': None, **ADL_metricas})
    
    # KNN
    knn_hyperpar = [
        {'n_neighbors': 3, 'weights': 'uniform'},
        {'n_neighbors': 5, 'weights': 'distance'}
    ]
    mejor_configuracion_knn = evalua_config(KNeighborsClassifier, knn_hyperpar, 5, X_train, y_train)
    knn_model = KNeighborsClassifier(**mejor_configuracion_knn)
    knn_metricas = evalua_metodo(knn_model, X_train, y_train, X_test, y_test)
    results.append({'Modelo': 'KNN', 'Configuración': mejor_configuracion_knn, **knn_metricas})
    
#Por ultimo, convertimos los resultados a una base de datos 
    df_results = pd.DataFrame(results)
    
    return df_results


